In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error
from math import sqrt
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import CoClustering, Dataset, Reader
from surprise.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error




## using movielens small dataset




In [ ]:
df1 = pd.read_csv(r"C:\Users\91798\Desktop\flame\FSP\courses'\ML\ml-latest-small\ml-latest-small\ratings.csv")
df1
df1.set_index(['userId', 'movieId'])
df1.info()

In [ ]:
df2 = pd.read_csv(r"C:\Users\91798\Desktop\flame\FSP\courses'\ML\ml-latest-small\ml-latest-small\movies.csv")
df2.set_index('movieId')
df2.head()

In [ ]:
# code to create a genres dataframe 
genresdf = pd.DataFrame()
# finding all genres of each movie and creating genres dictionary
for index, r in df2.iterrows():
    genre_dict = {  "movieId": r['movieId'],
                    "title": r['title'],
                    "Action": False,
                    "Adventure": False,
                    "Animation": False,
                    "Children": False,
                    "Comedy": False,
                    "Crime": False,
                    "Documentary": False,
                    "Drama": False,
                    "Fantasy": False,
                    "Film-Noir": False,
                    "Horror": False,
                    "Musical": False,
                    "Mystery": False,
                    "Romance": False,
                    "Sci-Fi": False,
                    "Thriller": False,
                    "War": False,
                    "Western": False,
                 "IMAX":False}
    
    genres = r['genres'].split("|")
    if "(no genres listed)" not in genres:
        for gen in genres:
            genre_dict[gen] = True
    # appending dictionary values of each row to genres dataframe
    genresdf = genresdf.append(genre_dict, ignore_index=True)

In [ ]:
genresdf

In [ ]:
# Now adding ratings dataframe and movie genres dataframe into a single dataframe
df= df1.merge(genresdf,  on='movieId')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()
#no missing values

In [ ]:
display(df.describe().T)


In [ ]:
df['Crime'].describe()

In [ ]:
(df['rating']).describe().T

## Using KNN

In [ ]:

# Standardize the ratings
#scaler = StandardScaler()
#df['rating'] = scaler.fit_transform(df['rating'].values.reshape(-1, 1))
# Pivot the data to have users as rows and movies as columns
df_pivot = df.pivot(index='userId', columns='movieId', values='rating').fillna(0)




In [ ]:
# Train the KNN model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(df_pivot)



In [ ]:
# Get the indices of the k most similar users
k = 10 # Change this to the desired number of recommendations
user_index = 1 # Change this to the desired user ID
distances, indices = model_knn.kneighbors(df_pivot.iloc[user_index-1,:].values.reshape(1, -1), n_neighbors=40)

# Get the movie recommendations for the user
movie_ids = df_pivot.columns[indices.flatten()[1:]]
movie_ratings = df_pivot.iloc[user_index-1,:][movie_ids]
movie_recommendations = pd.DataFrame({'movieId': movie_ids, 'predicted_rating': movie_ratings.values})

# Print the top 10 movie recommendations for the user
print('Top {} movie recommendations for user {}:'.format(k, user_index))
print(movie_recommendations.head(10))



In [ ]:

# Get the predictions for all users
distances, indices = model_knn.kneighbors(df_pivot)
all_predictions = []
for i in range(len(df_pivot)):
    if len(indices[i]) > 1:
        user_ratings = df_pivot.iloc[indices[i][1:],:]
        user_predictions = user_ratings.mean(axis=0)
    else:
        user_predictions = pd.Series(np.zeros_like(df_pivot.iloc[0]), index=df_pivot.columns)
    all_predictions.append(user_predictions)

# Calculate the RMSE
rmse = sqrt(mean_squared_error(df_pivot.values.flatten(), pd.concat(all_predictions).values))
print('RMSE:', rmse)

## SVD-Singular value decomposition

In [ ]:

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

# Split the dataset into train and test sets
trainset = data.build_full_trainset()

# Create an SVD algorithm instance
algo = SVD(n_epochs=25, verbose=True)

# Fit the algorithm to the training set
algo.fit(trainset)



In [ ]:
# Get the top 10 movie recommendations for user 1
user_id = 1
movie_ids = [i for i in range(1,df['movieId'].nunique()+1)]
movies_rated_by_user = df[df['userId'] == user_id]['movieId']
movie_ids_to_pred = list(set(movie_ids)-set(movies_rated_by_user))

predicted_ratings = []
for movie_id in movie_ids_to_pred:
    predicted_rating = algo.predict(str(user_id), str(movie_id)).est
    predicted_ratings.append((movie_id, predicted_rating))

recommended_movies = sorted(predicted_ratings, key=lambda x: x[1], reverse=True)[:10]
recommended_movies = df[df['movieId'].isin([movie[0] for movie in recommended_movies])][['movieId', 'title']].drop_duplicates().set_index('movieId').loc[[movie[0] for movie in recommended_movies]]['title']
print(recommended_movies)



In [ ]:
# Evaluate the algorithm using RMSE and MAE
results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Print the mean RMSE and MAE scores across all folds
print("Mean RMSE score:", round(results['test_rmse'].mean(), 3))
print("Mean MAE score:", round(results['test_mae'].mean(), 3))


## Co-clustering

In [ ]:


# Define the reader for the Surprise library
reader = Reader(rating_scale=(0.5, 5.0))

# Load the dataset into the Surprise format
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)



In [ ]:
# Train the Co-clustering model
model_cocluster = CoClustering()
model_cocluster.fit(trainset)

# Get the top k movie recommendations for a given user
k = 10 # Change this to the desired number of recommendations
user_id = 20 # Change this to the desired user ID

# Get the movies that the user has already rated
user_movies = df[df['userId'] == user_id]['movieId'].tolist()

# Get the movies that the user has not rated
anti_testset = trainset.build_anti_testset()
user_anti_testset = filter(lambda x: x[0] == user_id, anti_testset)
predictions = model_cocluster.test(user_anti_testset)



In [ ]:
# Sort the predictions by predicted rating in descending order
top_k = sorted(predictions, key=lambda x: x.est, reverse=True)[:k]

# Print the top k movie recommendations for the user
print('Top {} movie recommendations for user {}:'.format(k, user_id))
for i, prediction in enumerate(top_k):
    movie_id = prediction.iid
    title = df[df['movieId'] == movie_id]['title'].values[0]
    predicted_rating = prediction.est
    print('{}. {} ({:.2f})'.format(i+1, title, predicted_rating))
    
    
predictions = model_cocluster.test(testset)
rmse = accuracy.rmse(predictions)
print('RMSE score for Co-clustering model: {:.4f}'.format(rmse))

## Neural networks

In [ ]:
# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)
# Combine the data
combined = pd.concat([train, test], axis=0)

# Encode the user IDs
user_encoder = LabelEncoder()
combined['userId'] = user_encoder.fit_transform(combined['userId'])

# Encode the movie titles
title_encoder = LabelEncoder()
combined['movieId'] = title_encoder.fit_transform(combined['title'])

# Split the combined data back into training and test sets
train = combined.iloc[:train.shape[0], :]
test = combined.iloc[train.shape[0]:, :]



In [ ]:
# Scale the ratings
scaler = StandardScaler()
train['rating'] = scaler.fit_transform(train['rating'].values.reshape(-1, 1))
test['rating'] = scaler.transform(test['rating'].values.reshape(-1, 1))

# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(loss='mse', optimizer=Adam(lr=0.001))

# Define the early stopping callback
early_stopping = EarlyStopping(patience=5, verbose=1)



In [ ]:
# Train the model
history = model.fit(train[['userId', 'movieId']], train['rating'], epochs=50, batch_size=128, validation_split=0.1, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss = model.evaluate(test[['userId', 'movieId']], test['rating'])
test_rmse = np.sqrt(test_loss)
print('Test RMSE score: {:.4f}'.format(test_rmse))

# Predict the ratings on the test set
test['predicted_rating'] = scaler.inverse_transform(model.predict(test[['userId', 'movieId']]))
test['movie_title'] = test['title']

# Group the predicted ratings by movie title and calculate the average rating
avg_ratings = test[['movie_title', 'predicted_rating']].groupby('movie_title').mean().reset_index()

# Print the recommended movies
print('Recommended movies:')
for _, row in avg_ratings.sort_values(by='predicted_rating', ascending=False).iloc[:10].iterrows():
    print('{} ({:.4f})'.format(row['movie_title'], row['predicted_rating']))